<a href="https://colab.research.google.com/github/Futaba-Kosuke/cat_camera_ml/blob/issue%2F%235/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Install ngrok


In [1]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2020-12-09 01:48:15--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 3.225.137.4, 52.1.26.21, 52.55.159.231, ...
Connecting to bin.equinox.io (bin.equinox.io)|3.225.137.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  17.7MB/s    in 0.7s    

2020-12-09 01:48:16 (17.7 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


## 2. Execute ngrok

In [1]:
import time
get_ipython().system_raw('./ngrok http 3000 &')
time.sleep(1)
!curl -s http://localhost:4040/api/tunnels | python3 -c \
  "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://2ec0013394c7.ngrok.io


## 3. Run API Server

### 1. 各種設定



In [2]:
!nvidia-smi

Wed Dec  9 02:02:50 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!mkdir firebase
!mkdir models

mkdir: cannot create directory ‘firebase’: File exists
mkdir: cannot create directory ‘models’: File exists


In [4]:
from flask import Flask, request, jsonify, send_file, make_response, send_from_directory

import shutil
import os, glob
import base64
import json

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import cv2
from PIL import Image

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder

import torch
from torch import nn, optim

import torchvision
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader, Subset, WeightedRandomSampler

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [6]:
net_names = ('densenet',)
labels = []

In [7]:
import tempfile
import requests

def imread_web(url):
    # 画像をリクエストする
    res = requests.get(url)
    img = None
    # Tempfileを作成して即読み込む
    with tempfile.NamedTemporaryFile(dir='./') as fp:
        fp.write(res.content)
        fp.file.seek(0)
        img = cv2.imread(fp.name)
    return img

In [8]:
# 学習データを整形Tensorにして返す
def get_datas_and_labels(data_list):
    img_list_np = []
    labels = []

    i = 0
    for data in data_list:
        img_tmp = imread_web(data['url'])
        for cat_box in data['cat_boxes']:
            x_min, x_max, y_min, y_max = int(cat_box['x_min']), int(cat_box['x_max']), int(cat_box['y_min']), int(cat_box['y_max'])
            img_crop = img_tmp[y_min: y_max, x_min: x_max]
            img_list_np.append(cv2.resize(img_crop, (200, 200)))
            labels.append(data['label'])
            i += 1

    img_list_np = np.asarray(img_list_np)

    X = img_list_np.astype(np.float32)
    y = LabelEncoder().fit_transform(labels)

    return X, y, labels

# 層化交差検証でデータを分割
def stratified_fold(X, y, val_split=0.25):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    for train_idx, test_idx in skf.split(X, y):
        X_train, y_train = X[train_idx], y[train_idx]
        X_test, y_test = X[test_idx], y[test_idx]
    return (X_train, y_train), (X_test, y_test)

# 入力データと出力ラベルからDataLoaderを生成
def get_data_loader(X, y, batch_size):
    transform = torchvision.transforms.Compose([
        transforms.Resize((200, 200)),
        transforms.RandomHorizontalFlip(p=0.5),
        torchvision.transforms.ToTensor(),
    ])
    dataset = MyDataset(X, y, transform=transform)
    data_loader = DataLoader(dataset, batch_size=batch_size)
    return data_loader

In [9]:
# 自分用データセットを作成するクラス
class MyDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = data
        self.targets = torch.LongTensor(targets)
        self.transform = transform

    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]

        if self.transform:
            x = Image.fromarray(self.data[index].astype(np.uint8))
            x = self.transform(x)

        return x, y

    def __len__(self):
        return len(self.data)

In [10]:
# 学習用関数
def train_net(net, criterion, optimizer, scheduler, num_epochs, data_loaders, target_acc=100):
    for epoch in range(num_epochs):
        print('Epoch %d/%d' % (epoch, num_epochs - 1))
        print('-' * 10)

        running_loss_sum = 0
        train_loss_sum = 0
        test_loss_sum = 0

        train_cnt = 0
        test_cnt = 0
        cnt_total = 0

        # モデルの更新
        net.train()
        for i, (inputs, labels) in enumerate(data_loaders['train']):
            print(type(inputs))
            print(inputs.size())
            inputs = inputs.to(device).cuda()
            labels = labels.to(device).cuda()
            # 勾配の初期化
            optimizer.zero_grad()
            # 予測
            outputs = net(inputs)
            # 損失の導出
            loss = criterion(outputs, labels)
            # 逆伝播
            loss.backward()
            # 勾配の更新
            optimizer.step()

            running_loss_sum += loss.item()
            train_loss_sum += loss.item()
            train_cnt += 1
            if i % 50 == 0 and not i == 0:
                print('[%d] running_loss: %.6f' % (i + 1, running_loss_sum / 50))
                running_loss_sum = 0

        # モデルの評価
        with torch.no_grad():
            net.eval()
            cnt_correct = 0
            group_cnt = [0, 0, 0]
            for i, (inputs, labels) in enumerate(data_loaders['test']):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # 予測
                outputs = net(inputs)
                # 損失の導出
                loss = criterion(outputs, labels)
                # lossの加算
                test_loss_sum += loss.item()
                test_cnt += 1

                _, predicted = torch.max(outputs, 1)
                is_correct = (predicted == labels).squeeze()
                group_cnt[0] += np.count_nonzero(labels.to('cpu').detach().numpy().copy() == 0)
                group_cnt[1] += np.count_nonzero(labels.to('cpu').detach().numpy().copy() == 1)
                group_cnt[2] += np.count_nonzero(labels.to('cpu').detach().numpy().copy() == 2)
                
                for j in range(len(is_correct)):
                    cnt_correct += is_correct[j].item()  # 正解なら1, 不正解なら0
                    cnt_total += 1

            print('labels: ', group_cnt)
            print('train_loss_ave:\t%.6f' % (train_loss_sum / train_cnt))
            print('test_loss_ave:\t%.6f' % (test_loss_sum / test_cnt))
            print('test_accuracy: \t%.6f %%' % (100 * cnt_correct / (cnt_total)))

            net.train()

        scheduler.step()

        if (100 * cnt_correct / (cnt_total)) > target_acc:
            print('finish: %.6f' % (100 * cnt_correct / (cnt_total)))
            break

In [11]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

import datetime
import platform

class Firebase:

    def __init__(self, cred_path='./firebase/cred.json', bucket_name='cat-camera-cafe8.appspot.com'):
        cred = credentials.Certificate(cred_path)
        firebase_admin.initialize_app(cred, {'storageBucket': bucket_name})

        self.bucket_name = bucket_name
        self.bucket = storage.bucket()

        self.db = firestore.client()

    def upload_model(self, model_path='./models/densenet.pth', labels=[], meta_data={'from': platform.system()}):
        blob = self.bucket.blob(model_path.split('/')[-1])
        with open(model_path, 'rb') as f:
            blob.upload_from_file(f)
        blob.metadata = meta_data
        blob.patch()

        model_name = os.path.splitext(os.path.basename(model_path))[0]

        blob.make_public()
        url = blob.public_url
        self.db.collection('model').document(model_name).update(
            {
                'url': url,
                'is_enable': True,
                'is_update': True,
                'labels': labels
            }
        )

### 2. 学習モデルの設定

In [12]:
batch_size = 10
num_workers = 2

num_epochs = 5

In [13]:
# 学習モデルの選定
densenet = models.densenet161(pretrained=True)
last_in_features = densenet.classifier.in_features
densenet.classifier = nn.Linear(last_in_features, 3)
densenet = densenet.to(device)

nets = {
    'densenet': densenet
}

nets

{'densenet': DenseNet(
   (features): Sequential(
     (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
     (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (relu0): ReLU(inplace=True)
     (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
     (denseblock1): _DenseBlock(
       (denselayer1): _DenseLayer(
         (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
         (relu1): ReLU(inplace=True)
         (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
         (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
         (relu2): ReLU(inplace=True)
         (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       )
       (denselayer2): _DenseLayer(
         (norm1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_run

In [14]:
# 損失・最適関数の定義
from torch import optim

criterion = {
    i: nn.CrossEntropyLoss()
    for i in net_names
}
optimizer = {
    i: optim.SGD(nets[i].parameters(), lr=0.001, momentum=0.9)
    for i in net_names
}
scheduler = {
    i: optim.lr_scheduler.StepLR(optimizer[i], step_size=3, gamma=0.1)  # 10エポックごとに学習率が1/10に更新
    for i in net_names
}

### 3. Flaskの起動

In [15]:
app = Flask(__name__)

@app.route('/', methods=['POST'])
def hello():
    return {'a': 'a'}

@app.route('/training', methods=['POST'])
def training():

    print('train')

    data = request.data.decode('utf-8')
    data = json.loads(data)

    data_list = data['data_list']

    # Firebaseの設定
    cred = data['cred']
    cred_path = './firebase/cred.json'
    with open(cred_path, 'w') as f:
        json.dump(cred, f, indent=4)
    firebase = Firebase(cred_path=cred_path)

    # 渡されたdata_listから入力データX, 出力データyを整形
    X, y, labels = get_datas_and_labels(data_list)
    print(labels)

    # 層化交差検証でデータを分割
    (X_train, y_train), (X_test, y_test) = stratified_fold(X, y)

    # DataLoaderの作成
    data_loaders = {}
    data_loaders['train'] = get_data_loader(X_train, y_train, batch_size=batch_size)
    data_loaders['test']  = get_data_loader(X_test,  y_test,  batch_size=len(X_test))

    # 学習の実行
    net_name = 'densenet'
    train_net(
        net = nets[net_name],
        criterion = criterion[net_name],
        optimizer = optimizer[net_name],
        scheduler = scheduler[net_name],
        num_epochs = num_epochs,
        data_loaders = data_loaders,
    )

    model_dir = 'models'
    for net_name in net_names:
        model_path = os.path.join(model_dir, '{}.pth'.format(net_name))
        torch.save(nets[net_name].state_dict(), model_path)

        firebase.upload_model(model_path=model_path, labels=list(set(labels)))

    # 後処理
    del firebase
    os.remove(cred_path)

    return '200'

if __name__ == '__main__':
    app.run(port=3000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:3000/ (Press CTRL+C to quit)


train
['つくし', 'つくし', 'つくし', 'つくし', 'つくし', 'つくし', 'つくし', 'つくし', 'つくし', 'つくし', 'つくし', 'つくし', 'つくし']
Epoch 0/4
----------
<class 'torch.Tensor'>
torch.Size([10, 3, 200, 200])
<class 'torch.Tensor'>
torch.Size([1, 3, 200, 200])
labels:  [2, 0, 0]
train_loss_ave:	0.972847
test_loss_ave:	0.565267
test_accuracy: 	100.000000 %
Epoch 1/4
----------
<class 'torch.Tensor'>
torch.Size([10, 3, 200, 200])
<class 'torch.Tensor'>
torch.Size([1, 3, 200, 200])
labels:  [2, 0, 0]
train_loss_ave:	0.327329
test_loss_ave:	0.265093
test_accuracy: 	100.000000 %
Epoch 2/4
----------
<class 'torch.Tensor'>
torch.Size([10, 3, 200, 200])
<class 'torch.Tensor'>
torch.Size([1, 3, 200, 200])
labels:  [2, 0, 0]
train_loss_ave:	0.065486
test_loss_ave:	0.144102
test_accuracy: 	100.000000 %
Epoch 3/4
----------
<class 'torch.Tensor'>
torch.Size([10, 3, 200, 200])
<class 'torch.Tensor'>
torch.Size([1, 3, 200, 200])
labels:  [2, 0, 0]
train_loss_ave:	0.014028
test_loss_ave:	0.134743
test_accuracy: 	100.000000 %
Epoch 4/4


127.0.0.1 - - [09/Dec/2020 02:03:09] "POST /training HTTP/1.1" 200 -
